In [ ]:
#run these commands when you start the program
get_ipython().system('pip install mediapipe opencv-python')
get_ipython().system('pip install numpy scipy Sphinx numpydoc nose pykalman')
get_ipython().system('pip install keyboard')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 28.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 26.5 MB/s eta 0:00:00
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=0af8271914e5ed290f2a6f227e976a07dbf39b734327b95d95ab644ddf5d6e8f
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully buil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#program imports
import mediapipe as mp
import cv2
import numpy as np
import os
import sys
import imageio
import keyboard
%matplotlib notebook
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
from google.colab.patches import cv2_imshow

In [ ]:
# Rename some packages for ease of use
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

In [ ]:
# Name of input video file and extension
name = "HANDYA034_Center_Pinch_RH_1"
extension = ".mpeg"

In [ ]:
# Define some variables
input_video_path    = ''
output_video_path   = ''
output_csv_path     = ''
output_smooth_path  = ''

In [ ]:
# if paths weren't explicitly stated, paths are assumed
if input_video_path == "":
    input_video_path = f'/content/drive/MyDrive/Mediapipe_Folders/input_videos/HandYA034/{name}{extension}'
if output_video_path == "":
    output_video_path = f'/content/drive/MyDrive/Mediapipe_Folders/output_videos/HandYA034_Output/{name}.avi'
if output_csv_path == "":
    output_csv_path = f'/content/drive/MyDrive/Mediapipe_Folders/csv/{name}.csv'
if output_smooth_path == "":
    output_smooth_path = f'/content/drive/MyDrive/Mediapipe_Folders/smooth_csv/{name}_smooth.csv'

print("Path of input video: ",input_video_path)
print("------------------------------------------")
if (os.path.exists(input_video_path)):
    print("Input video exists, continue with program!")
    print("------------------------------------------")
else:
    print("file not found, please correct the path!")
    print("------------------------------------------")
if (os.path.exists(output_video_path)):
    print("There already exists a video of this name")
    print("Are you sure you want to overwrite it?")
    print("------------------------------------------")

if (os.path.exists(output_csv_path)):
    print("There already exists a csv of this name")
    print("Are you sure you want to overwrite it?")
    print("------------------------------------------")

# Pay attention to output below

Path of input video:  /content/drive/MyDrive/Mediapipe_Folders/input_videos/HandYA034/HANDYA034_Center_Pinch_RH_1.mpeg
------------------------------------------
Input video exists, continue with program!
------------------------------------------


In [ ]:
# open video in opencv and initialize output video
# cap = cv2.VideoCapture(0) #webcam for testing
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_input = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # you can adjust codec depending on os, preference, etc.
print("Input Video is: " + str(frame_width) + " by " + str(frame_height) + " at " + str(fps_input) + " FPS")
out = cv2.VideoWriter(output_video_path, fourcc, fps_input, (frame_width, frame_height))
# might give you some warnings about a codec not supported
# not very important, but it may be difficult for others to play the video with VLC

Input Video is: 1920 by 1080 at 59 FPS


In [ ]:
# define landmarks and csv columns
landmark_names_face = ['NOSE', 'LEFT_EYE_IN', 'LEFT_EYE', 'LEFT_EYE_OUT', 'RIGHT_EYE_IN', 'RIGHT_EYE', 'RIGHT_EYE_OUT',
                       'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT']
landmark_names_body = ['LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST',
                       'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB',
                       'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
                       'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
landmark_names_pose = np.concatenate([landmark_names_face, landmark_names_body])

POSE_LANDMARK_INDICES = [11, 12, 13, 14, 15, 16]
POSE_LANDMARK_NAMES = [landmark_names_pose[i] for i in POSE_LANDMARK_INDICES]

# Generate landmark names
landmark_names_pose = POSE_LANDMARK_NAMES
landmark_names_left = ['LEFT_WRIST_HAND', 'LEFT_THUMB_1', 'LEFT_THUMB_2', 'LEFT_THUMB_3', 'LEFT_THUMB_4',
                       'LEFT_INDEX_1', 'LEFT_INDEX_2', 'LEFT_INDEX_3', 'LEFT_INDEX_4',
                       'LEFT_MIDDLE_1', 'LEFT_MIDDLE_2', 'LEFT_MIDDLE_3', 'LEFT_MIDDLE_4',
                       'LEFT_RING_1', 'LEFT_RING_2', 'LEFT_RING_3', 'LEFT_RING_4',
                       'LEFT_PINKY_1', 'LEFT_PINKY_2', 'LEFT_PINKY_3', 'LEFT_PINKY_4']
landmark_names_right = ['RIGHT_WRIST_HAND', 'RIGHT_THUMB_1', 'RIGHT_THUMB_2', 'RIGHT_THUMB_3', 'RIGHT_THUMB_4',
                        'RIGHT_INDEX_1', 'RIGHT_INDEX_2', 'RIGHT_INDEX_3', 'RIGHT_INDEX_4',
                        'RIGHT_MIDDLE_1', 'RIGHT_MIDDLE_2', 'RIGHT_MIDDLE_3', 'RIGHT_MIDDLE_4',
                        'RIGHT_RING_1', 'RIGHT_RING_2', 'RIGHT_RING_3', 'RIGHT_RING_4',
                        'RIGHT_PINKY_1', 'RIGHT_PINKY_2', 'RIGHT_PINKY_3', 'RIGHT_PINKY_4']
landmark_names_all = np.hstack([landmark_names_pose, landmark_names_left, landmark_names_right])

# Adds in the _X _Y _Z to all the landmark names
landmark_names_pose = [name + suffix
                       for name in landmark_names_pose
                       for suffix in ['_X', '_Y', '_Z']]
landmark_names_left = [name + suffix
                       for name in landmark_names_left
                       for suffix in ['_X', '_Y', '_Z']]
landmark_names_right = [name + suffix
                        for name in landmark_names_right
                        for suffix in ['_X', '_Y', '_Z']]
landmark_names_all = [name + suffix
                      for name in landmark_names_all
                      for suffix in ['_X', '_Y', '_Z']]

len_landmarks_all = len(landmark_names_all)
landmark_names_all = np.array(landmark_names_all)
landmark_names_all.shape = (1, len_landmarks_all)

len_landmarks_all = int(len_landmarks_all / 3)

print("number of points:", len_landmarks_all)

# initializing data arrays
data_left_len = int(len(landmark_names_left) / 3) # Should be 21
data_right_len = int(len(landmark_names_right) / 3) # Should be 21
data_pose_len = int(len(landmark_names_pose) / 3) # Should be  6

data_left = np.zeros(shape=(1, data_left_len, 3))  # Left Hand Data
data_right = np.zeros(shape=(1, data_right_len, 3))  # Right Hand Data
data_pose = np.zeros(shape=(1, data_pose_len, 3))  # Pose Data

number of points: 48


In [ ]:
# Rename more packages and some drawing connections
POSE_CONNECTIONS = mp.solutions.pose.POSE_CONNECTIONS
HAND_CONNECTIONS = mp.solutions.hands_connections.HAND_CONNECTIONS

VALID_POSE_CONNECTIONS = []
for a, b in POSE_CONNECTIONS:
    if a in POSE_LANDMARK_INDICES and b in POSE_LANDMARK_INDICES:
        VALID_POSE_CONNECTIONS.append((a, b))
POSE_CONNECTIONS = VALID_POSE_CONNECTIONS

In [ ]:
# This function will run the Kalman Smoothing we will use later
# Some parameters can be changed to make the smoothing more similar to what is desired
def smooth(arr, initial_state_mean=0, b=-1):
    intermed = arr.swapaxes(0, 2)
    iter_total = intermed.shape[0] * intermed.shape[1]
    iter = 1
    for i in range(intermed.shape[0]):
        for j in range(intermed.shape[1]):
            kf = KalmanFilter(initial_state_mean=initial_state_mean, n_dim_obs=1)
            measurements = np.ma.masked_invalid(intermed[i][j])
            if b > -1:
                kf = kf.em(measurements, n_iter=b)
            else:
                kf = kf.em(measurements)
            smooth_meas = kf.smooth(measurements)[0]
            intermed[i][j] = smooth_meas.T[0]
            sys.stdout.write(f'\rFinished smoothing point {iter} of {iter_total}')
            sys.stdout.flush()
            iter += 1

    return intermed.swapaxes(0, 2)

In [ ]:
# Main mediapipe program block
iter = 0
missed = 0
BREAK = False

# may give some warnings about "QObject" but thats because pyqt5 and opencv do not get along
# just ignore it, there is not much we can do to fix it nor is it really a problem
with mp_pose.Pose(
        min_detection_confidence=0.6,
        min_tracking_confidence=0.6,
        model_complexity=2) as pose:  # calibrate detection and tracking confidence for best results
    with mp_hands.Hands(min_detection_confidence=0.6,
                        min_tracking_confidence=0.6,
                        max_num_hands=2) as hands:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Tracking complete, " + str(iter) + ' frames analyzed')
                print(f"Percent Missed detections: %.2f%%" % ((missed / iter) * 100))
                print("Your tracked output video will be saved to " + output_video_path)
                print("Your output .csv data file will be saved to " + output_csv_path)
                print("Your smoothed output .csv data file will be saved to " + output_smooth_path)
                break

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # image = cv2.flip(image, 1)

            image.flags.writeable = False

            results_pose = pose.process(image)
            results_hand = hands.process(image)

            # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            POSE_DETECTED = results_pose.pose_world_landmarks

            L_HAND = (False, -1)
            R_HAND = (False, -1)
            if results_hand.multi_hand_world_landmarks:
                num_hands = len(results_hand.multi_hand_landmarks)
                for hand_landmarks in results_hand.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, HAND_CONNECTIONS)
                for i in range(num_hands):
                    handedness = results_hand.multi_handedness[i]
                    if handedness.classification[0].label == "Left":  # Right hand
                        R_HAND = (True, i)
                    if handedness.classification[0].label == "Right":  # Left hand
                        L_HAND = (True, i)

            POSE_WRIST_L = 15
            POSE_WRIST_R = 16

            # If left hand is detected
            if L_HAND[0]:
                data_temp = [[nl.x, nl.y, nl.z]
                             for nl in results_hand.multi_hand_world_landmarks[L_HAND[1]].landmark]
                data_left = np.vstack([data_left, [data_temp]])
            else:

                # Fill Left Hand array with NaN if not detected
                data_nan = np.empty(shape=(1, data_left_len, 3))
                data_nan[:] = np.NaN
                data_left = np.vstack([data_left, data_nan])

            # If right hand detected
            if R_HAND[0]:
                data_temp = [[nl.x, nl.y, nl.z]
                             for nl in results_hand.multi_hand_world_landmarks[R_HAND[1]].landmark]

                data_right = np.vstack([data_right, [data_temp]])
            else:

                # Fill Right Hand array with NaN if not detected
                data_nan = np.empty(shape=(1, data_right_len, 3))
                data_nan[:] = np.NaN
                data_right = np.vstack([data_right, data_nan])

            if (POSE_DETECTED):
                wanted_landmarks = [results_pose.pose_world_landmarks.landmark[i] for i in POSE_LANDMARK_INDICES]
                data_temp = [[nl.x, nl.y, nl.z]
                             for nl in wanted_landmarks]
                # iterate and save the landmarks to array
                data_pose = np.vstack([data_pose, [data_temp]])

                # Prioritize Hand's Wrist point over Pose's Wrist since Hand is more accurate
                # Modify Pose points so that we can use MP's built-in utility to draw points
                # We are still saving Pose's Wrist data point, but we would rather draw the Hand's wrist data point
                m_pose_landmarks = results_pose.pose_landmarks

                # Hide unwanted landmarks
                for i in range(len(m_pose_landmarks.landmark)):
                    if i not in POSE_LANDMARK_INDICES:
                        m_pose_landmarks.landmark[i].visibility = 0

                if L_HAND[0]:
                    left_hand_wrist = results_hand.multi_hand_landmarks[L_HAND[1]].landmark[0]
                    left_pose_wrist = m_pose_landmarks.landmark[POSE_WRIST_L]

                    left_pose_wrist.x = left_hand_wrist.x
                    left_pose_wrist.y = left_hand_wrist.y
                    left_pose_wrist.z = left_hand_wrist.z
                if R_HAND[0]:
                    right_hand_wrist = results_hand.multi_hand_landmarks[R_HAND[1]].landmark[0]
                    right_pose_wrist = m_pose_landmarks.landmark[POSE_WRIST_R]

                    right_pose_wrist.x = right_hand_wrist.x
                    right_pose_wrist.y = right_hand_wrist.y
                    right_pose_wrist.z = right_hand_wrist.z


                mp_drawing.draw_landmarks(
                    image,
                    m_pose_landmarks,
                    POSE_CONNECTIONS)

            else:
                print("no detection pose")
                missed = missed + 1  # Missed variable only updates when pose is not detected (change?)

                # Fill pose array with NaN if not detected
                data_nan = np.empty(shape=(1, data_pose_len, 3))
                data_nan[:] = np.NaN
                data_pose = np.vstack([data_pose, data_nan])

            iter += 1
            out.write(image)
            #cv2_imshow(image)  # must flip image

            if cv2.waitKey(1) == ord('q'):  # press the "q" key to stop the tracking
                # Will cause the csv to not output and numpy will throw
                # an error on the next block if you stopped the video
                BREAK = True
                break;

cap.release()
out.release()
cv2.destroyAllWindows()

if BREAK:
    exit(2)

no detection pose
Tracking complete, 1430 frames analyzed
Percent Missed detections: 0.07%
Your tracked output video will be saved to /content/drive/MyDrive/Mediapipe_Folders/output_videos/HandYA034_Output/HANDYA034_Center_Pinch_RH_1.avi
Your output .csv data file will be saved to /content/drive/MyDrive/Mediapipe_Folders/csv/HANDYA034_Center_Pinch_RH_1.csv
Your smoothed output .csv data file will be saved to /content/drive/MyDrive/Mediapipe_Folders/smooth_csv/HANDYA034_Center_Pinch_RH_1_smooth.csv


In [ ]:
packaged_data = np.hstack([data_pose, data_left, data_right])
packaged_data = np.delete(packaged_data, 0, axis=0)  # Remove first "row" of placeholder data

packaged_data.shape = (-1, packaged_data.shape[1] * packaged_data.shape[2])

# save
print(f"\nSaving data to {output_csv_path}")
with open(output_csv_path, 'a', newline='') as csvfile:  # output csv
    np.savetxt(output_csv_path, landmark_names_all, fmt='%s', delimiter=',')
    np.savetxt(csvfile, packaged_data, fmt='%f', delimiter=',')



Saving data to /content/drive/MyDrive/Mediapipe_Folders/csv/HANDYA034_Center_Pinch_RH_1.csv


In [ ]:
# # Define some paths for the visualizer related to where we saved the previous files
# input_csv_path = 'drive/MyDrive/colab_data/smooth_csv/dext_front_smooth.csv' 
# input_avi_path = 'drive/MyDrive/colab_data/output_videos/dext_front.avi'
# FPS = 30 #you can adjust this to make the visualizer faster

# if not os.path.exists(input_avi_path):
#     print("Input video not found")
#     exit()


In [ ]:
# # Take in the mediapipe video to have a reference for the visualizer
# cap = cv2.VideoCapture(input_avi_path)
# frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps_input = int(cap.get(cv2.CAP_PROP_FPS))
# fourcc = cv2.VideoWriter_fourcc(*'XVID')  # you can adjust codec depending on os, preference, etc.

In [ ]:
# # NUMBER OF LANDMARKS
# L_NUM, R_NUM, P_NUM = 21, 21, 6

In [ ]:
# # Connections between hands
# HAND_CONNECTIONS = mp.solutions.hands_connections.HAND_CONNECTIONS
# POSE_CONNECTIONS = mp.solutions.pose.POSE_CONNECTIONS

# POSE_INDICES = [11, 12, 13, 14, 15, 16]     # These are the indices of the points captured
# pose_dict = dict(zip(POSE_INDICES, range(len(POSE_INDICES))))   # Convert indices to indices of pose array

In [ ]:
# # Map the visible connections to indices of the pose_data array
# VALID_CONNECTIONS = []
# for a, b in POSE_CONNECTIONS:
#     if a in POSE_INDICES and b in POSE_INDICES:
#         VALID_CONNECTIONS.append((pose_dict[a], pose_dict[b]))

# POSE_CONNECTIONS = VALID_CONNECTIONS


In [ ]:
# # Data is formatted as (Pose, Left hand, Right hand)
# with open(input_csv_path, 'r') as f:
#     data = np.genfromtxt(f, delimiter=',', skip_header=1, dtype=float)
#     data.shape = (-1, int(data.shape[1] / 3), 3)
#     frames = data.shape[0]

# split_indices = [P_NUM, L_NUM + P_NUM]       # Split array back into sub parts
# pose_data, left_data, right_data = np.hsplit(data, split_indices)

In [ ]:
# # This block created the visualizer
# # This takes a very long time close to 30 min occasionally
# # It is very inefficient, but that is because of notebook style environments
# # Loop initializations
# do_save = True
# figures = []

# cap = cv2.VideoCapture(input_avi_path)

# for frame in range(frames):
#     # Set new points and line locations
#     if cap.isOpened():
#         success, image = cap.read()
#         #cv2_imshow(image)
#     # Init plot, probably not necessary to do this repeatly
#     # But, it does not work without it
#     plt.ion()
#     fig = plt.figure(figsize=(50,50))
#     ax = fig.add_subplot(111, projection='3d')
#     ax.set_xlabel("x")
#     ax.set_ylabel("y")
#     ax.set_zlabel("z")
#     ax.set_xlim(-.5, .5)
#     ax.set_ylim(-.5, .5)
#     ax.set_zlim(-.5, .5)
#     ax.set_title("Hand markers with z relative to wrist")
#     ax.view_init(elev=210, azim=90)
#     # Change elev and azim to change the viewing angle of the 3-D plot
#     # May be necessary for different videos
#     temp = []
#     temp.extend(HAND_CONNECTIONS)
#     temp.extend(HAND_CONNECTIONS)
#     temp.extend(POSE_CONNECTIONS)

#     lines = [ax.plot([], [], [], color = "green")[0]
#          for line in temp]
#     scatters = ax.scatter([], [], [], color = "red")

#     i = 0
#     current_pose = pose_data[frame]
#     current_left = left_data[frame]
#     current_right = right_data[frame]
#     L = current_left[0]
#     R = current_right[0]
#     current_left[:] += current_pose[4] - L
#     current_right[:] += current_pose[5] - R
#     for a, b in POSE_CONNECTIONS:
#         lines[i].set_data_3d([current_pose[a][2], current_pose[b][2]],
#                              [current_pose[a][0], current_pose[b][0]],
#                              [current_pose[a][1], current_pose[b][1]])
#         i += 1

#     for a, b in HAND_CONNECTIONS:
#         lines[i].set_data_3d([current_left[a][2], current_left[b][2]],
#                              [current_left[a][0], current_left[b][0]],
#                              [current_left[a][1], current_left[b][1]])
#         i += 1
#         lines[i].set_data_3d([current_right[a][2], current_right[b][2]],
#                              [current_right[a][0], current_right[b][0]],
#                              [current_right[a][1], current_right[b][1]])
#         i += 1
#     ditto = np.vstack([current_pose, current_left, current_right])
#     scatters._offsets3d = (ditto.T[2], ditto.T[0], ditto.T[1])

#     # Update graph
#     plt.draw()
#     fig.canvas.flush_events()
#     filename = f"{frame}.png"
#     if frame == 0:
#         cv2.waitKey(1000000)
#     plt.savefig(filename)
#     figures.append(filename)
#     plt.pause(1 / FPS)
#     plt.close(fig)
# if do_save:
#     name = 'drive/MyDrive/colab_data/visualizer_videos/' + name + 'smooth_viz.mp4' 
#     # Will save file as an mp4 to the drive
#     with imageio.get_writer(name, mode='I') as writer:
#         for filename in figures:
#             image = imageio.imread(filename)
#             writer.append_data(image)
# os.system("rm *.png") # Removes from temporary static images we created